In [1]:
import os
import tensorflow as tf
import sys
import urllib
import numpy as np
import pdb
import dataset

if sys.version_info[0] >= 3:
  from urllib.request import urlretrieve
else:
  from urllib import urlretrieve

LOGDIR = 'skin/'
# Number of color channels for the images: 1 channel for gray-scale.
num_channels = 3

# image dimensions (only squares for now)
img_size = 128

# Size of image when flattened to a single dimension
img_size_flat = img_size * img_size * num_channels

# Tuple with height and width of images used to reshape arrays.
img_shape = (img_size, img_size)

# class info

classes = ['benign', 'malignant']
num_classes = len(classes)

# batch size
batch_size = 40
train_batch_size = batch_size
# validation split
validation_size = .3

# Add convolution layer
def conv_layer(input, size_in, size_out, name="conv"):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
    act = tf.nn.relu(conv + b)
    return tf.nn.max_pool(act, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


# Add fully connected layer
def fc_layer(input, size_in, size_out, name="fc"):
  with tf.name_scope(name):
    w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
    act = tf.matmul(input, w) + b
    return act
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()

    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features
def print_progress(sess,epoch, feed_dict_train, feed_dict_validate, val_loss,accuracy):
    # Calculate the accuracy on the training-set.
    acc = sess.run(accuracy, feed_dict=feed_dict_train)
    val_acc = sess.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%}, Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

def skin_model(learning_rate, use_two_conv, use_two_fc, hparam):
  tf.reset_default_graph()
  config = tf.ConfigProto()
  config.gpu_options.allocator_type = 'BFC'
  sess = tf.Session(config = config)

  # Setup placeholders, and reshape the data
  x = tf.placeholder(tf.float32, shape=[None, 49152], name="x")
  x_image = tf.reshape(x, [-1, img_size, img_size, num_channels])
  tf.summary.image('input', x_image, 3)
  y = tf.placeholder(tf.float32, shape=[None, 2], name="labels")

  if use_two_conv:
    conv1 = conv_layer(x_image, num_channels, 32, "conv1")
    conv_out = conv_layer(conv1, 32, 64, "conv2")
  else:
    conv1 = conv_layer(x_image, num_channels, 64, "conv")
    conv_out = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

  flattened, num_features = flatten_layer(conv_out)


  if use_two_fc:
    fc1 = fc_layer(flattened, num_features, 128, "fc1")
    logits = fc_layer(fc1, 128, 2, "fc2")
    tf.add_to_collection("logits", logits)
  else:
    logits = fc_layer(flattened, img_size_flat, 2, "fc")
    tf.add_to_collection("logits", logits)

  with tf.name_scope("xent"):
    xent = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, labels=y), name="xent")
    xent_summary = tf.summary.scalar("xent", xent)

  with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(xent)

  with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    train_summary = tf.summary.scalar("train_accuracy", accuracy)
    validate_summary = tf.summary.scalar("validate_accuracy", accuracy)




  sess.run(tf.global_variables_initializer())
  writer = tf.summary.FileWriter(LOGDIR + hparam)
  writer.add_graph(sess.graph)

  config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
 
  train_path='/home/mati/Documentos/Harvard/bigdata/finalproject/dataset/ISIC-Dataset-Downloader/train_small'

  data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)

  print("Size of:")
  print("- Training-set:\t\t{}".format(len(data.train.labels)))
  print("- Validation-set:\t{}".format(len(data.valid.labels)))


  # step 4: Batching
  #image_batch = tf.train.batch([resized_image], batch_size=8)
  for i in range(6000):
    # Get a batch of training examples.
    # x_batch now holds a batch of images and
    # y_true_batch are the true labels for those images.
    x_batch, y_true_batch, _, cls_batch = data.train.next_batch(train_batch_size)
    x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(train_batch_size)
       
    # Convert shape from [num examples, rows, columns, depth]
    # to [num examples, flattened image shape]
    x_batch = x_batch.reshape(train_batch_size, img_size_flat)
    x_valid_batch = x_valid_batch.reshape(train_batch_size, img_size_flat)
    # Put the batch into a dict with the proper names
    # for placeholder variables in the TensorFlow graph.
    feed_dict_train = {x: x_batch,
                           y: y_true_batch}
        
    feed_dict_validate = {x: x_valid_batch,
                              y: y_valid_batch}
    if i % 5 == 0:
      [train_accuracy,train_sum,xent_sum] = sess.run([accuracy,train_summary,xent_summary], feed_dict=feed_dict_train)
      writer.add_summary(train_sum, i)
      writer.add_summary(xent_sum, i)
    # Print status at end of each epoch (defined as full pass through training dataset).
    if i % int(data.train.num_examples/batch_size) == 0: 
        val_loss = sess.run(xent, feed_dict=feed_dict_validate)
        epoch = int(i / int(data.train.num_examples/batch_size))
        print_progress(sess,epoch, feed_dict_train, feed_dict_validate, val_loss,accuracy)
        [validate_accuracy,validate_sum] = sess.run([accuracy,validate_summary], feed_dict=feed_dict_validate)
        writer.add_summary(validate_sum, i)
        
    sess.run(train_step, feed_dict=feed_dict_train)
  model_saver = tf.train.Saver() 
  # Train the model and save it in the end
  model_saver.save(sess, os.path.join(os.getcwd(), 'skin.ckpt'))
  model_saver.export_meta_graph(os.path.join(os.getcwd(), 'skin.meta'))  
    
def make_hparam_string(learning_rate, use_two_fc, use_two_conv):
  conv_param = "conv2" if use_two_conv else "conv1"
  fc_param = "fc2" if use_two_fc else "fc1"
  return "lr_%.0E%s%s" % (learning_rate, conv_param, fc_param)
def main():
  # You can try adding some more learning rates
  #for learning_rate in [1E-3, 1E-4, 1E-5]:
  for learning_rate in [1E-4]:

    # Include "False" as a value to try different model architectures
    #for use_two_fc in [True, False]:
    for use_two_fc in [True]:
      #for use_two_conv in [True, False]:
      for use_two_conv in [True]:
        # Construct a hyperparameter string for each one (example: "lr_1E-3fc2conv2")
        hparam = make_hparam_string(learning_rate, use_two_fc, use_two_conv)
        print('Starting run for %s' % hparam)
        sys.stdout.flush() # this forces print-ed lines to show up.

	    # Actually run with the new settings
        skin_model(learning_rate, use_two_fc, use_two_conv, hparam)


if __name__ == '__main__': 
  main()

Starting run for lr_1E-04conv2fc2
Reading training images
Loading benign files (Index: 0)
Loading malignant files (Index: 1)
Size of:
- Training-set:		1894
- Validation-set:	811
Epoch 1 --- Training Accuracy:  35.0%, Validation Accuracy:  32.5%, Validation Loss: 14.998
Epoch 2 --- Training Accuracy:  75.0%, Validation Accuracy:  75.0%, Validation Loss: 0.645
Epoch 3 --- Training Accuracy:  72.5%, Validation Accuracy:  77.5%, Validation Loss: 0.470
Epoch 4 --- Training Accuracy:  70.0%, Validation Accuracy:  70.0%, Validation Loss: 0.795
Epoch 5 --- Training Accuracy:  72.5%, Validation Accuracy:  85.0%, Validation Loss: 0.536
Epoch 6 --- Training Accuracy:  80.0%, Validation Accuracy:  70.0%, Validation Loss: 0.743
Epoch 7 --- Training Accuracy:  87.5%, Validation Accuracy:  62.5%, Validation Loss: 0.879
Epoch 8 --- Training Accuracy:  85.0%, Validation Accuracy:  70.0%, Validation Loss: 0.617
Epoch 9 --- Training Accuracy:  72.5%, Validation Accuracy:  57.5%, Validation Loss: 0.978
Ep